## Подготовка

Импортируем необходимые для работы библиотеки:

In [1]:
import pandas as pd
import numpy as np
import re 
import tqdm
import nltk
import spacy
from tqdm import notebook
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import f1_score

Откроем файл (так как проект был выполнен локально, путь к файлу прописан для той же папки, где находился файл проекта):

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col = 0).reset_index(drop = True)
print(df.head())
df.info()

                                                text  toxic
0  Explanation\nWhy the edits made under my usern...      0
1  D'aww! He matches this background colour I'm s...      0
2  Hey man, I'm really not trying to edit war. It...      0
3  "\nMore\nI can't make any real suggestions on ...      0
4  You, sir, are my hero. Any chance you remember...      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Проверим данные на дубликаты:

In [3]:
df['text'].duplicated().sum()

0

Создадим функцию, определяющую тег слова для WordNetLemmatizer:

In [4]:
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Создадим функцию, производящую токенизацию и лемматизацию текста:

In [5]:
def lemmatize(text):
    
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))
    wordnet_tagged = list(map(lambda index: (index[0], pos_tagger(index[1])), pos_tagged))
    
    lemmatized_sentence = []
    
    for word, tag in wordnet_tagged:
        
        if tag is None:
            lemmatized_sentence.append(word)
        else:
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
            
    lemmatized_sentence = " ".join(lemmatized_sentence)
        
    return lemmatized_sentence

Лемматизируем датасет:

In [6]:
df['lemm_text'] = df['text'].copy()

for index in notebook.tqdm(range(len(df['text']))):
    df['lemm_text'][index] = lemmatize(df['text'][index].lower())

  0%|          | 0/159292 [00:00<?, ?it/s]

/tmp/ipykernel_537/3167603441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemm_text'][index] = lemmatize(df['text'][index].lower())


In [7]:
df.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww ! he match this background colour i 'm s...
2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , i 'm really not try to edit war . it..."
3,"""\nMore\nI can't make any real suggestions on ...",0,`` more i ca n't make any real suggestion on i...
4,"You, sir, are my hero. Any chance you remember...",0,"you , sir , be my hero . any chance you rememb..."


## Обучение

### Подготовка модели

Разделим датасет на обучающие и тестовые `features` и `target`:

In [8]:
features_train, features_test, target_train, target_test = train_test_split(df['lemm_text'], 
                                                                            df['toxic'], shuffle = False, test_size = 0.2)

Загрузим стоп-слова английского языка и подготовим `features` к обучению с помощью `TF-IDF`: 

In [9]:
stopwords = set(nltk_stopwords.words('english'))

tf_idf_model = TfidfVectorizer(stop_words = stopwords)
tf_idf_train = tf_idf_model.fit_transform(features_train.values)
tf_idf_test = tf_idf_model.transform(features_test.values)

### Модель RF

Проверим качество модели, обучив ее на алгоритме RF. Основной метрикой будет `F1`:

In [10]:
for est in range(1, 5):
    for depth in notebook.tqdm(range(1, 5)):
        
        model_RF = RandomForestClassifier(random_state = 12345, n_estimators = est, 
                                          max_depth = depth,  max_features = 10_000)
        model_RF.fit(tf_idf_train, target_train)
        
        scores_RF = cross_val_score(model_RF, tf_idf_train, target_train, scoring = 'f1', cv = 5)
        final_score_RF = sum(scores_RF) / len(scores_RF)
        
        print('Значение F1-метрики:', final_score_RF)
        
        if final_score_RF >= 0.75:  
            
            print('Значение гиперпараметра max_depth:', depth)    
            print('Значение гиперпараметра n_estimators:', est)  
            break
    break

  0%|          | 0/4 [00:00<?, ?it/s]

Значение F1-метрики: 0.13859692548416466
Значение F1-метрики: 0.2086117324078586
Значение F1-метрики: 0.2535800637572451
Значение F1-метрики: 0.2922309499209972


Из полученных результатов следует, что модель RF не выполняет задачу с заданной точностью

### Модель LR

Проверим качество модели, обучив ее на алгоритме LR. Основной метрикой будет `F1`:

In [11]:
for index in notebook.tqdm(range(35, 40)):

    model_LR = LogisticRegression(random_state = 12345, C = index / 10, max_iter = 1000)
    model_LR.fit(tf_idf_train, target_train)
    
    scores_LR = cross_val_score(model_LR, tf_idf_train, target_train, scoring = 'f1', cv = 5)
    final_score_LR = sum(scores_LR) / len(scores_LR)

    if final_score_LR >= 0.75:  
        
        print('Значение F1-метрики:', final_score_LR)
        print('Значение гиперпараметра C:', index / 10)      
        break

  0%|          | 0/5 [00:00<?, ?it/s]

Значение F1-метрики: 0.7518631134149449
Значение гиперпараметра C: 3.5


Из полученных результатов следует, что модель RF выполняет задачу с заданной точностью (F1 не менее 0.75)

### Тестирование

Проведем тестирование модели RF и определим, как меняется точность модели на тестовой выборке, используя лучшие гиперпараметры:

In [12]:
model_LR_test = LogisticRegression(random_state = 12345, C = 3.5, max_iter = 1000)
model_LR_test.fit(tf_idf_train, target_train)
predictions_RF_test = model_LR_test.predict(tf_idf_test)
print('Значение F1-метрики:', f1_score(target_test, predictions_RF_test))

Значение F1-метрики: 0.7608461814270349


Значение на тестовой выборке не ухудшилось, модель работает с заданной точностью

## Выводы

В ходе выполнения проекта были произведены: подготовка данных, обучение на разных моделях и тестирование.

В процессе выполнения подготовки данных данные были проведены анализ на дубликаты, анализ на пропуски, исходный текст был лемматизирован с помощью `WordNetLemmatizer`.

В процессе обучения модели были произведены обучение на моделях RF и LR. Получены следующие результаты:
1. Модель RF не работает с заданной точностью
2. Модель LR имеет качество модели по метрике F1 0.750866, что подходит под необходимое условие (не менее 0.75)

Тестирование производилось для модели LR. Результат - качество модели немного выше (0.764568), чем на обучающей вборке, модель работает с заданной точностью.

Таким образом можно сделать вывод о том, что качество модели обучения текста в TF-IDF + LR гораздо лучше, чем у TF-IDF + RF